In [2]:
from collections import defaultdict
from statistics import mean
import glob
import json

def marginalize_complementizer(lang2score):
    lang2marginalized_score = defaultdict(list)
    for lang, score in lang2score.items():
        lang2marginalized_score[lang[:2] + "X" + lang[3:]].append(float(score))
    lang2score = {}
    for lang, scores in lang2marginalized_score.items():
        lang2score[lang] = mean(scores)
    return lang2score

In [3]:
from statistics import mean

def reduce(stack, action):
    for i, a  in enumerate(stack[::-1], start=1):
        if a.startswith("NT"):
            assert a[3:4] == action[7:8]
            break
    stack = stack[:-i] + ["(" + "_".join(stack[-i:]) + ")"]
    return stack

def lc_as(line):
    stack = []
    stack_ls = []
    for action in line.split():
        if action.startswith("NT"):
            stack = stack[:-1] + [action + "_" + stack[-1]]
        elif action.startswith("REDUCE"):
            stack = reduce(stack, action)
        else:
            stack.append(action)
        stack_ls.append(len(stack))
    return stack_ls

def top_down(line):
    stack = []
    stack_ls = []
    for action in line.split():
        if action.startswith("NT"):
            stack.append(action)
        elif action.startswith("REDUCE"):
            stack = reduce(stack, action)
        else:
            stack[-1] = stack[-1] + "_" + action
        stack_ls.append(len(stack))
    return stack_ls


In [8]:
folds = ["0", "20000", "40000", "60000", "80000"]
for fold in folds:
    lang2stack_d = {}
    files = glob.glob(f"../work/tree_per_line/**/{fold}/lc-as/tst.actions.txt")
    for file in files:
        lang = file.split("/")[-4]
        stack_ds = []
        with open(file) as f:
            for line in f:
                stack_ds.extend(lc_as(line))
        lang2stack_d[lang] = mean(stack_ds)
    lang2stack_d = marginalize_complementizer(lang2stack_d)
    json.dump(lang2stack_d, open(f"../work/results/lang_stack_depth/lc-as_fold{fold}.json", "w"))

In [9]:
folds = ["0", "20000", "40000", "60000", "80000"]
for fold in folds:
    lang2stack_d = {}
    files = glob.glob(f"../work/tree_per_line/**/{fold}/td/tst.actions.txt")
    for file in files:
        with open(file) as f:
            lang = file.split("/")[-4]
            stack_ds = []
            for line in f:
                stack_ds.extend(top_down(line))
            lang2stack_d[lang] = mean(stack_ds)
    lang2stack_d = marginalize_complementizer(lang2stack_d)
    json.dump(lang2stack_d, open(f"../work/results/lang_stack_depth/td_fold{fold}.json", "w"))